<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 8.5 - Prompting Large Language Models

In this lab we will practise prompting with a few Large Language Models (LLMs) using Groq (not to be confused with Grok). Groq is a platform that provides access to their custom-built AI hardware via APIs, allowing users to run open-source models such as Llama.

We shall see that while LLMs are powerful tools, how you ask a question or frame a task can dramatically influence the results obtained.

## Set-up

Step 1: Sign up for a free Groq account at https://console.groq.com/home .

Step 2: Create a new API key at https://console.groq.com/keys. Copy-paste it into an empty text file called 'groq_key.txt'.

Running the next cell will then read in this key and assign it to the variable `groq_key`.

In [1]:
groqfilename = r'groq_key.txt' # this file contains a single line containing your Groq API key only
try:
    with open(groqfilename, 'r') as f:
        groq_key = f.read().strip()
except FileNotFoundError:
    print(f"'{groqfilename}' file not found")

In [3]:
#!pip install groq

In [4]:
from groq import Groq
import requests
import pandas as pd
from IPython.display import Markdown

First create an instance of the Groq client:

In [6]:
client = Groq(api_key=groq_key)

The following code shows what models are currently accessible through Groq. `context_window` refers to the size of memory (in tokens) during a session and `max_completion_tokens` is the maximum number of tokens that are generated in an output.

In [8]:
url = "https://api.groq.com/openai/v1/models"

headers = {
    "Authorization": f"Bearer {groq_key}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

pd.DataFrame(response.json()['data']).sort_values(['created'], ascending=False)

,id,object,created,owned_by,active,context_window,public_apps,max_completion_tokens
11,openai/gpt-oss-120b,model,1754408224,OpenAI,True,131072,None,65536
21,openai/gpt-oss-20b,model,1754407957,OpenAI,True,131072,None,65536
0,moonshotai/kimi-k2-instruct,model,1752435491,Moonshot AI,True,131072,None,16384
5,meta-llama/llama-prompt-guard-2-86m,model,1748632165,Meta,True,512,None,512
8,meta-llama/llama-prompt-guard-2-22m,model,1748632101,Meta,True,512,None,512
19,qwen/qwen3-32b,model,1748396646,Alibaba Cloud,True,131072,None,40960
9,meta-llama/llama-guard-4-12b,model,1746743847,Meta,True,131072,None,1024
15,meta-llama/llama-4-maverick-17b-128e-instruct,model,1743877158,Meta,True,131072,None,8192
16,meta-llama/llama-4-scout-17b-16e-instruct,model,1743874824,Meta,True,131072,None,8192
17,compound-beta-mini,model,1742953279,Groq,True,131072,None,8192


The Groq client object enables interaction with the Groq REST API and a chat completion request is made via the client.chat.completions.create method.

The most important arguments of the client.chat.completions.create method are the following:
* messages: a list of messages (dictionary form) that make up the conversation to date
* model: a string indicating which model to use (see [list of models](https://console.groq.com/docs/models))
* max_completion_tokens: the maximum number of tokens that are generated in the chat completion
* response_format: setting this to `{ "type": "json_object" }` enables JSON output
* seed: sample deterministically as best as possible, though identical outputs each time are not guaranteed
* temperature: between 0 and 2 where higher values like 0.8 make the output more random (creative) and values like 0.2 are more focused and deterministic


In [10]:
help(client.chat.completions.create)

Help on method create in module groq.resources.chat.completions:

create(*, messages: 'Iterable[ChatCompletionMessageParam]', model: "Union[str, Literal['gemma2-9b-it', 'llama-3.3-70b-versatile', 'llama-3.1-8b-instant', 'llama-guard-3-8b', 'llama3-70b-8192', 'llama3-8b-8192']]", exclude_domains: 'Optional[List[str]] | NotGiven' = NOT_GIVEN, frequency_penalty: 'Optional[float] | NotGiven' = NOT_GIVEN, function_call: 'Optional[completion_create_params.FunctionCall] | NotGiven' = NOT_GIVEN, functions: 'Optional[Iterable[completion_create_params.Function]] | NotGiven' = NOT_GIVEN, include_domains: 'Optional[List[str]] | NotGiven' = NOT_GIVEN, include_reasoning: 'Optional[bool] | NotGiven' = NOT_GIVEN, logit_bias: 'Optional[Dict[str, int]] | NotGiven' = NOT_GIVEN, logprobs: 'Optional[bool] | NotGiven' = NOT_GIVEN, max_completion_tokens: 'Optional[int] | NotGiven' = NOT_GIVEN, max_tokens: 'Optional[int] | NotGiven' = NOT_GIVEN, metadata: 'Optional[Dict[str, str]] | NotGiven' = NOT_GIVEN, n: 

As a first example, note how the messages input is given as a list of a dictionaries with `role` and `content` keys. This is in a ChatML format recognised by many LLMs.

In [13]:
chat_completion = client.chat.completions.create(
    messages=[
        {   "role": "system", # sets the persona of the model
            "content": "You are a helpful assistant."
        },
        {
            "role": "user", # what the user wants the assistant to do
            "content": "Explain briefly how large language models work",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Large language models are artificial intelligence (AI) systems that process and generate human-like language. Here's a brief overview of how they work:

1. **Training**: These models are trained on massive amounts of text data, often from the internet, books, or other sources.
2. **Pattern recognition**: The models use complex algorithms to recognize patterns in the training data, such as grammar, syntax, and word relationships.
3. **Neural network**: The models are built using neural networks, which are composed of layers of interconnected nodes (neurons) that process and transform the input data.
4. **Tokenization**: When you input text, it's broken down into individual tokens (words, characters, or subwords).
5. **Generation**: The model uses the patterns learned during training to predict the next token, given the context of the input text.
6. **Iteration**: This process is repeated, with the model generating text one token at a time, until a complete response is formed.

By levera

The output is in Markdown format so the following line formats this text.

In [15]:
Markdown(chat_completion.choices[0].message.content)

Large language models are artificial intelligence (AI) systems that process and generate human-like language. Here's a brief overview of how they work:

1. **Training**: These models are trained on massive amounts of text data, often from the internet, books, or other sources.
2. **Pattern recognition**: The models use complex algorithms to recognize patterns in the training data, such as grammar, syntax, and word relationships.
3. **Neural network**: The models are built using neural networks, which are composed of layers of interconnected nodes (neurons) that process and transform the input data.
4. **Tokenization**: When you input text, it's broken down into individual tokens (words, characters, or subwords).
5. **Generation**: The model uses the patterns learned during training to predict the next token, given the context of the input text.
6. **Iteration**: This process is repeated, with the model generating text one token at a time, until a complete response is formed.

By leveraging these techniques, large language models can understand and generate coherent, context-dependent text, allowing them to perform tasks like answering questions, translating languages, and even creating original content.

## Text summarisation

We start with a llama3-8b-8192, a model using just over 8 billion parameters with at most 8192 tokens produced as output.

Here is an article to be summarised from the [cnn_dailymail](https://huggingface.co/datasets/abisee/cnn_dailymail) dataset:

In [20]:
story = """
SAN FRANCISCO, California (CNN) -- A magnitude 4.2 earthquake shook the San Francisco area Friday at 4:42 a.m. PT (7:42 a.m. ET), the U.S. Geological Survey reported. The quake left about 2,000 customers without power, said David Eisenhower, a spokesman for Pacific Gas and Light. Under the USGS classification, a magnitude 4.2 earthquake is considered "light," which it says usually causes minimal damage. "We had quite a spike in calls, mostly calls of inquiry, none of any injury, none of any damage that was reported," said Capt. Al Casciato of the San Francisco police. "It was fairly mild." Watch police describe concerned calls immediately after the quake » . The quake was centered about two miles east-northeast of Oakland, at a depth of 3.6 miles, the USGS said. Oakland is just east of San Francisco, across San Francisco Bay. An Oakland police dispatcher told CNN the quake set off alarms at people's homes. The shaking lasted about 50 seconds, said CNN meteorologist Chad Myers. According to the USGS, magnitude 4.2 quakes are felt indoors and may break dishes and windows and overturn unstable objects. Pendulum clocks may stop.
"""

**Exercise:**
Summarise the story text using the following three prompts. Use the format given above but here there is no need to set the persona (i.e. only include one dictionary in the messages list when calling `client.chat.completions.create`.) Comment on any differences.

1) "Summarise the following article in 3 sentences."

2) "Give me a TL;DR of this text."

3) "What's the key takeaway here?"

In [22]:
prompts = ["Summarise the following article in 3 sentences. ", "Give me a TL;DR of this text. ", "What's the key takeaway here?"]
#content will be p + story for p in prompts

# ANSWER
for p in prompts:
    response = client.chat.completions.create(
        model = "llama3-8b-8192",
        messages = [{"role": "user", "content": p+story}]
    )
    
    print(p, "\n", response.choices[0].message.content)

Summarise the following article in 3 sentences.  
 Here is a summary of the article in 3 sentences:

A magnitude 4.2 earthquake struck the San Francisco area on Friday morning, causing minimal damage and only temporarily leaving about 2,000 customers without power. The quake was considered "light" and was felt for about 50 seconds, with no reported injuries or significant damage. According to the USGS, a magnitude 4.2 quake can cause items to fall and break, but overall, the impact was described as "fairly mild" by authorities.
Give me a TL;DR of this text.  
 Here is a TL;DR (Too Long; Didn't Read) summary of the text:

A 4.2-magnitude earthquake struck the San Francisco area at 4:42am on Friday, causing about 2,000 power outages and triggering alarm calls. The quake was shallow (3.6 miles deep) and centered near Oakland, about 2 miles east of San Francisco. The USGS considers 4.2 magnitudes as "light" and minimal damage was reported.
What's the key takeaway here? 
 The key takeaway i

Run the above code again below and note that the answers may differ. This is due to the probabilistic nature of LLM token generation.

In [25]:
# ANSWER
prompts = ["Summarise the following article in 3 sentences. ", "Give me a TL;DR of this text. ", "What's the key takeaway here?"]
#content will be p + story for p in prompts

# ANSWER
for p in prompts:
    response = client.chat.completions.create(
        model = "llama3-8b-8192",
        messages = [{"role": "user", "content": p+story}]
    )
    
    print(p, "\n", response.choices[0].message.content)

Summarise the following article in 3 sentences.  
 Here is a summary of the article in 3 sentences:

A magnitude 4.2 earthquake struck the San Francisco area at 4:42am PT, causing minimal damage and no reported injuries. The quake was centered 2 miles east-northeast of Oakland, just east of San Francisco, and was described as "fairly mild" by authorities. The earthquake left about 2,000 customers without power, but overall damage was minimal, with only a few reports of knocked-over objects and alarm clocks set off in homes.
Give me a TL;DR of this text.  
 Here is a brief summary of the text:

A magnitude 4.2 earthquake struck the San Francisco area at 4:42am, causing minimal damage and no reported injuries. About 2,000 customers lost power, but most call-ins were inquiries rather than reports of damage. The quake was centered near Oakland, lasted around 50 seconds, and caused some objects to fall or be disturbed.
What's the key takeaway here? 
 The key takeaway is that a magnitude 4.2

## Text completion

**Exercise**: In this section adjust the `max_completion_tokens` and `temperature` settings below to obtain different responses. Show some examples with the prompt "Continue the story: It was a great time to be alive" with the model "llama-3.1-8b-instant".

* max_completion_tokens - the maximum number of tokens to generate. Note that longer words are made of multiple tokens (set to 200 and 500)
* temperature (positive number) - the higher the number the more random (creative) the output (set to 0.2, 0.8, 2)

In [30]:
# ANSWER (set max_completion_tokens=200, do not have a temperature setting)

response = client.chat.completions.create(
    model = "llama-3.1-8b-instant",
    messages = [{"role": "user", "content": "Continue the story: It was a great time to be alive"}],
    max_completion_tokens = 200
)
    
print(response.choices[0].message.content)

As I walked through the bustling streets of the city, I couldn't help but feel a sense of excitement and joy that seemed to permeate the air. Everywhere I looked, there were people smiling, laughing, and soaking up the sun. The atmosphere was electric, and I found myself grinning from ear to ear as I took it all in.

It was the summer of 1967, and I was 25 years old. I had just returned from a few months traveling the world, and I was settled back into my routine as a young journalist. I had always been passionate about music, and I felt like I was at the epicenter of it all.

Every night, I would head to the legendary Fillmore Auditorium in San Francisco, where the likes of Janis Joplin, Jimi Hendrix, and The Grateful Dead would take the stage. The music was like nothing I had ever heard before - raw, emotional, and deeply connected to the times we were living


In [31]:
# ANSWER (set max_completion_tokens=500, do not have a temperature setting)

response = client.chat.completions.create(
    model = "llama-3.1-8b-instant",
    messages = [{"role": "user", "content": "Continue the story: It was a great time to be alive"}],
    max_completion_tokens = 500
)
    
print(response.choices[0].message.content)

It was a great time to be alive, with the world at the brink of a new era of technological advancement. The air was electric with the hum of innovation, as humans reached for the stars and made giant leaps towards a brighter future.

Ava sat on the edge of her bed, her fingers dancing across the sleek surface of her holographic screen as she scrolled through the latest news feeds. She watched in awe as a new breakthrough in space exploration was announced, marking the first time humans had successfully established a colony on Mars.

Her city, New Eden, pulsed with excitement as the news spread like wildfire. People poured out of their homes and apartments, gathering in the city square to celebrate the occasion. Ava joined the throngs, her eyes shining with wonder as she watched the Martian colonists being honored on the massive digital screens that lined the square.

As she gazed up at the screens, Ava felt a tap on her shoulder. She turned to see her best friend, Jax, grinning at her 

In [34]:
# ANSWER (set temperature = 0.2, do not have a max_completion_tokens setting)

response = client.chat.completions.create(
    model = "llama-3.1-8b-instant",
    messages = [{"role": "user", "content": "Continue the story: It was a great time to be alive"}],
    temperature = 0.2
)
    
print(response.choices[0].message.content)

It was a great time to be alive, and for Emily, it was especially so. The year was 1969, and the world was on the cusp of a revolution. The Beatles had just released their iconic album "Abbey Road," and the music scene was buzzing with the sounds of Woodstock and the Summer of Love.

Emily, a free-spirited 22-year-old, was right in the middle of it all. She had just graduated from college with a degree in art, and was working as a waitress at a trendy coffee shop in the Haight-Ashbury neighborhood of San Francisco. Her days were filled with the sounds of Jimi Hendrix and Janis Joplin, and her nights were spent dancing at the Fillmore with her friends.

One evening, as she was closing up the coffee shop, Emily met a young man named Max. He was a musician, with a guitar slung over his shoulder and a mop of curly hair that fell across his forehead. He had a charming smile and a quick wit, and Emily was immediately drawn to him.

As they talked, Emily learned that Max was part of a new ban

In [36]:
# ANSWER (set temperature = 1, do not have a max_completion_tokens setting)
response = client.chat.completions.create(
    model = "llama-3.1-8b-instant",
    messages = [{"role": "user", "content": "Continue the story: It was a great time to be alive"}],
    temperature = 1
)
    
print(response.choices[0].message.content)

It was a great time to be alive, and Emma couldn't help but feel a sense of euphoria wash over her as she walked through the sun-kissed streets of 1960s San Francisco. The air was alive with the sounds of jazz music drifting from the clubs on Fillmore Street, and the smell of freshly brewed coffee wafted through the air from the quaint cafes that lined the way.

As she strolled along, Emma's bright red dress catching the eye of every suited businessman who passed by, she felt a sense of freedom that she'd never known before. The war was over, and the world was finally beginning to open up to young people like her, full of hope and possibility.

Emma had always been a rebel, drawn to the music and the art of the Beat Generation. She'd spent countless hours listening to Allen Ginsberg and Jack Kerouac read their poetry at City Lights Books, and she'd even tried her hand at writing her own verse, though it was still a secret she kept hidden from the world.

As she turned a corner, Emma sp

Note what happens when the temperature is set too high!

In [38]:
# ANSWER (set temperature = 2, do not have a max_completion_tokens setting)
response = client.chat.completions.create(
    model = "llama-3.1-8b-instant",
    messages = [{"role": "user", "content": "Continue the story: It was a great time to be alive"}],
    temperature = 2
)
    
print(response.choices[0].message.content)

It was a great time to be alive, and that fact wasn't lost on Jack Harris, a carefree 22-year-old who had just graduated from a thriving art school in San Francisco's vibrant Bay Area. The world was on the precipice of a creative revolution, led in part by Jack and his fellow creatives who saw opportunity at every street corner of this rapidly growing and ever-changing tech-obsessed city.

The morning of the day we were getting started, the first rays peeks into Jack Harris's bright room on Potrero Hill at just before 05h00. That sun rise in such place was truly magical. But Jack could never sleep this morning, too excited. This city is full, always of life.


### Zero-shot and one-short prompting for question-answering

This section shows the impact of prompting on the response. Zero-shot prompting means we provide the prompt without any examples or additional context. Let us initially ask Mistral a question using no prompting.

In [42]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "How do two chemicals react?"}],
    temperature = 0.8,
)

Markdown(response.choices[0].message.content)

The reaction between two chemicals is determined by their chemical properties, such as their electronegativity, polarity, and reactivity. Here's a simplified overview of the process:

1. **Atoms interact**: When two chemicals come into contact, their atoms interact through intermolecular forces such as London dispersion forces, dipole-dipole forces, and hydrogen bonding.
2. **Molecules collide**: The atoms collide with each other, and if the collision is energetic enough, the molecules may break apart or react.
3. **Reaction occurs**: If the molecules have the correct arrangement of electrons and atoms, a chemical reaction can occur. This is determined by the laws of thermodynamics, such as the law of conservation of energy and the law of conservation of matter.
4. **Chemical bonds form or break**: During the reaction, chemical bonds are either formed or broken between the atoms. This can result in the creation of new compounds or the transformation of existing ones.
5. **Energy is released or absorbed**: As the reaction occurs, energy is either released (exothermic reaction) or absorbed (endothermic reaction).

Types of chemical reactions include:

1. **Combustion reactions**: Oxygen reacts with a fuel to produce heat and light.
Example: 2H2 + O2 → 2H2O
2. **Dispersion reactions**: A chemical is dispersed into a solvent, often resulting in a solution.
Example: NaCl + H2O → Na+ + Cl- + H2O
3. **Acid-base reactions**: An acid reacts with a base to form a salt and water.
Example: HCl + NaOH → NaCl + H2O
4. **Oxidation-reduction reactions**: A chemical loses or gains electrons, resulting in a change in oxidation state.
Example: 2Na + Cl2 → 2NaCl
5. **Synthesis reactions**: Two or more chemicals combine to form a new compound.
Example: 2H2 + O2 → 2H2O

These are just a few examples of chemical reactions. The specific reaction between two chemicals depends on their chemical properties and the conditions under which they interact.

**Factors influencing chemical reactions:**

1. **Temperature**: Higher temperatures often increase the rate and yield of chemical reactions.
2. **Pressure**: Increased pressure can increase the rate of some chemical reactions.
3. **Concentration**: Higher concentrations of the reacting substances can increase the rate and yield of the reaction.
4. **Catalysts**: Some substances can speed up or slow down chemical reactions without being consumed in the process.

**Tools for predicting chemical reactions:**

1. **Balancing equations**: Writing balanced chemical equations can help predict the products and stoichiometry of a reaction.
2. **Reaction mechanisms**: Understanding the step-by-step process of a reaction can help predict the products and conditions required for the reaction to occur.
3. **Thermodynamics**: Using thermodynamic principles, such as the Gibbs free energy equation, can help predict the spontaneity and direction of a reaction.

Keep in mind that predicting chemical reactions can be complex and depends on various factors, including the specific conditions and substances involved.

**Exercise:** Ask the same question but modify the prompt to return the answer to the same question in a simpler form (still using the llama-3.1-8b-instant model). Experiment with different prompts.

In [44]:
# ANSWER
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Explain how two chemicals react in very simple words, i am beginner"}],
    temperature=0.8,
)

display(Markdown(response.choices[0].message.content))

Let's say we have two chemicals: Chemical A and Chemical B.

**What happens when they meet?**

Imagine Chemical A and Chemical B are two friends who want to interact with each other. They might not know each other at first, but when they meet, they start to talk and share stuff.

**Chemical Reaction:**

When Chemical A and Chemical B meet, they can do two things:

1. **Be friends**: They can mix together and become a new, combined chemical. This is called a chemical reaction. It's like they've become a new team!
2. **Change each other**: They can change each other's shape or form. This is also a chemical reaction. It's like they're wearing each other's hats!

**Example:**

* Chemical A might be a liquid (like water).
* Chemical B might be a gas (like oxygen).
* When they meet, they can form a new chemical (like water vapor).
* Or, Chemical A might change into a solid (like ice) when it meets Chemical B.

**Simple Example:**

* Chemical A: Salt (sodium chloride)
* Chemical B: Water
* When they meet, they form a new chemical: Saltwater.

That's it! Chemical reactions are like interactions between friends. They can mix and become new things, or change each other's shape or form.

Do you have any questions?

### One-shot prompting ###

Next, note the dramatic change when we give the following template setting a new role and providing an English question followed by a French translation.

In [48]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "system",
             "content": "You translate English to French."},
              {"role": "user",
               "content": "What time is it?"},
               {"role": "assistant",
               "content": "Quelle heure est-il?"},
              {"role": "user",
               "content": "How do two chemicals react?"}],
    temperature = 0.8,
)
print(response.choices[0].message.content)

Comment deux composés chimiques réagissent-ils?


### Few-shot prompting

Recall that since the text generation process outputs one token at a time, their outputs often need adjusting. This is where examples can help.

In [52]:
prompt1 = "I'm gonna head out now, see you later."
response1 = "I will be leaving now. See you later."

prompt2 =  "That movie was super cool!"
response2 = "The movie was very impressive."

prompt3 = "Can't make it to the meeting, sorry."


response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are a professional editor. Rewrite casual sentences into a formal tone."},
        {"role": "user", "content": prompt1},
        {"role": "assistant", "content": response1},
        {"role": "user", "content": prompt2},
        {"role": "assistant", "content": response2},
        {"role": "user", "content": prompt3},
    ]
)

print(response.choices[0].message.content.strip())


Regrettably, I will be unable to attend the meeting. I apologize for any inconvenience this may cause.


The output can also be moulded to provide SQL output.

In [55]:
prompt1 = "Show me all users who signed up in the last 30 days."
response1 = "SELECT * FROM users WHERE signup_date >= CURRENT_DATE - INTERVAL '30 days';"

prompt2 = "What is the average order value?"
response2 =  "SELECT AVG(order_total) FROM orders;"

prompt3 = "List products that are out of stock."

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are an assistant that translates natural language to SQL."},
        {"role": "user", "content": prompt1},
        {"role": "assistant", "content": response1},
        {"role": "user", "content": prompt2},
        {"role": "assistant", "content": response2},
        {"role": "user", "content": prompt3},
    ]
)

print(response.choices[0].message.content.strip())


SELECT * FROM products WHERE quantity = 0;


**Exercise**: Create a few examples to train the "llama3-70b-8192" LLM to take in user content in the form below and provide output as a pandas dataframe. Use the `exec` function to execute its output to display the answer of sample input as a data frame.

Example:

given the user content

"""

| col1 | col2 | col3

| 32 | 27 | 25

| 64 | 23 | 14

"""

train the model to output

df = pd.DataFrame({'col1': [32, 64], 'col2': [27, 23], 'col3': [25, 14]})



In [64]:
#ANSWER
prompt1 = "take in user content and provide output as a pandas dataframe."
prompt2 = """

| col1 | col2 | col3 |

| 32 | 27 | 25 |

| 64 | 23 | 14 |

"""

prompt3 = """df = pd.DataFrame({'col1': [32, 64],
                   'col2': [27, 23],
                   'col3': [25, 14]})
"""

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": prompt1},
        {"role": "user", "content": prompt2},
        {"role": "assistant", "content": prompt3}
    ]
)

output = response.choices[0].message.content.strip()
print("Code generated:\n", output)

Code generated:
 print(df)


Output:

   col1  col2  col3
0    32    27    25
1    64    23    14


Also show what happens when the question is asked in the absence of a system role and without few-shot prompting.

In [67]:
# ANSWER
prompt1 = """ Change the data to dataframe

| col1 | col2 | col3 |

| 32 | 27 | 25 |

| 64 | 23 | 14 |

"""

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[{"role": "user", "content": prompt1}]
)

output = response.choices[0].message.content.strip()
print("Code generated:\n", output)

Code generated:
 You can use the `pd.DataFrame` constructor to create a Pandas DataFrame from the data:

```
import pandas as pd

data = [[32, 27, 25], [64, 23, 14]]
columns = ['col1', 'col2', 'col3']

df = pd.DataFrame(data, columns=columns)

print(df)
```

Output:

```
   col1  col2  col3
0    32    27    25
1    64    23    14
```

Alternatively, you can also use the `pd.DataFrame.from_records` method:

```
import pandas as pd

data = [[32, 27, 25], [64, 23, 14]]
columns = ['col1', 'col2', 'col3']

df = pd.DataFrame.from_records(data, columns=columns)

print(df)
```

Both methods will produce the same output.


### Chain-of-thought prompting

The results of question-answering can also be improved by prompting the LLM to provide intermediate steps.

**Exercise**: Using the following prompts, compare the answers of the "llama3-8b-8192" model (set seed=21). (If this model is no longer available choose a model with relatively few parameters.)

zero_shot_prompt = "How many s's are in the word 'success'?"

chain_of_thought_prompt = "How many s's are in the word 'success'? Explain your answer step by step by going through each letter in turn."

In [76]:
# ANSWER
prompt = [ 
    "How many s's are in the word 'success'?",
    "How many s's are in the word 'success'? Explain your answer step by step by going through each letter in turn."
]

model = "llama3-8b-8192"

results = []

# Looping each model + question
for p in prompt: 
        response = client.chat.completions.create(
            model= model,
            messages=[{"role": "user", "content": p}],
            max_tokens=300,
            seed = 21
        )
        
        answer = response.choices[0].message.content.strip()
        
        results.append({
            "Model Name": model,
            "Question": p,
            "Answer": answer
        })

df = pd.DataFrame(results)
print(df.to_string())

       Model Name                                                                                                        Question                                                                                                                                                                                                                                                                                                                  Answer
0  llama3-8b-8192                                                                         How many s's are in the word 'success'?                                                                                                                                                                                                                                                                                  There are 2 s's in the word "success".
1  llama3-8b-8192  How many s's are in the word 'success'? Explain your answer step by step by going through each le

## Comparison of LLMs

**Exercise**: Compare the performance of 2 LLMs by outputting the answers of the following questions into a dataframe.

    "Tell me a joke about data science.",
    "How can one calculate 22 * 13 mentally?",
    "Write a creative story about a baby learning to crawl.",

Column headings:

Model Name | Question | Answer

In [80]:
pd.set_option('display.max_colwidth', None) # allows wide dataframes to be viewed
models = ["gemma2-9b-it", "llama-3.1-8b-instant"] #can edit this

# ANSWER
prompt = [ 
    "Tell me a joke about data science.",
    "How can one calculate 22 * 13 mentally?",
    "Write a creative story about a baby learning to crawl."
]

results = []

# Looping each model + question
for model in models:
    for p in prompt: 
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": p}],
            max_tokens=300
        )
        
        answer = response.choices[0].message.content.strip()
        
        results.append({
            "Model Name": model,
            "Question": p,
            "Answer": answer
        })

df = pd.DataFrame(results)
print(df.to_string())

             Model Name                                                Question                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

### Bonus

See if you can prompt an LLM to perform sentiment analysis (output 'Positive' or 'Negative' only) on a given piece of text.

In [90]:
# ANSWER
prompt1 = ("Please help to classifier this statement based on sentiment analysis."
            "Do not output explanation and response only 'Positive or Negative'.")
prompt2 = ["I love to solve data science issue but i find difficulties to understand many techniques behind data science."]

model = "llama3-8b-8192"

rows = []
for t in prompt2:
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt1},
            {"role": "user", "content": t}
        ],
        temperature=0,
        max_completion_tokens=5,
    )
    sentiment = resp.choices[0].message.content.strip()
    rows.append({"Text": t, "Sentiment": sentiment})

df = pd.DataFrame(rows)
print(df)

                                                                                                            Text  \
0  I love to solve data science issue but i find difficulties to understand many techniques behind data science.   

  Sentiment  
0  Negative  


## Conclusion

We worked with a few Large Language Models (LLMs) using Groq and experimented with prompting for summarisation, text completion and question-answering tasks.

We also explored controlling the randomness (creativity) of output through the temperature setting and tried different types of prompting to achieve desired forms of output.

## References
1. [Groq's prompting guide](https://console.groq.com/docs/prompting)
2. [Groq's playground](https://console.groq.com/playground)



---



---



> > > > > > > > > © 2025 Institute of Data


---



---



